# <center> GEC
GEC pour tester les modèles et les métriques en sortie.
Implémentation de la métrique GLEU ?
Utilisation du modèle BART ? Ou de l'implémentation de BERT ? Ou de l'implémentation de GPT-2 ? Ou de l'implémentation de T5 ? Ou de l'implémentation de GPT-3 ?

Implémentation d'un modèle GEC en simulation de la pipeline.


Modèle T5:
https://huggingface.co/vennify/t5-base-grammar-correction

Alors, de ce que j'ai vu on peut utiliser des modèles text to text comme le modèle transformer T5 de google ou le modèle BART de facebook.
BART : https://ai.meta.com/research/publications/bart-denoising-sequence-to-sequence-pre-training-for-natural-language-generation-translation-and-comprehension/
T5 : https://blog.research.google/2020/02/exploring-transfer-learning-with-t5.html
En se référant au SOTA de la tâche GEC : on peut considérer les modèles transformer comme étant les meilleurs modèles pour la tâche GEC.
https://arxiv.org/pdf/2106.03830.pdf
https://aclanthology.org/2022.coling-1.246/
https://dl.acm.org/doi/abs/10.1145/3474840

https://scholar.google.com/scholar?hl=fr&as_sdt=0%2C5&q=gec+grammatical+error+correction&btnG=&oq=GEC

Aussi différents concours on eu lieu, comme le BEA :
https://sig-edu.org/bea/2019/ en 2019 le BEA aborde la tâche GEC. https://www.cl.cam.ac.uk/research/nl/bea2019st/
**https://competitions.codalab.org/competitions/20228#results**
https://aclanthology.org/W19-4406.pdf

SOTA 2020 : grammarly avec GECTor : https://arxiv.org/pdf/2005.12592v2.pdf   https://github.com/grammarly/gector
best SOTA ? : https://aclanthology.org/2022.naacl-main.143/


huggingface pretrained : https://huggingface.co/transformers/v3.3.1/pretrained_models.html

https://huggingface.co/docs/transformers/model_doc/t5

https://huggingface.co/docs/transformers/model_doc/bart
papiers bart : https://arxiv.org/abs/2005.11849

BTS TTT :  https://aclanthology.org/2021.wat-1.10.pdf

https://aclanthology.org/W19-4413.pdf
LLM GEC : https://aclanthology.org/2021.emnlp-main.611.pdf


BERT
https://sunilchomal.github.io/GECwBERT/
https://huggingface.co/bert-base-uncased


GPT3 https://aclanthology.org/2023.bea-1.18.pdf
https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo
https://arxiv.org/abs/2303.13648


GEC VIDEO (BANGER0): https://aclanthology.org/2022.naacl-main.143.mp4

___
## Imports libraries

In [23]:
from datasets import load_dataset
import torch, os
from torch.utils.data import DataLoader, Dataset

from tqdm.notebook import tqdm
from happytransformer import HappyTextToText, TTSettings

___
## Import des données

In [117]:
class ASR_Dataset(Dataset):
    def __init__(self, path):
        self.path = path
        self.files = os.listdir(path)
        self.sentences = []
        for file in self.files:
            with open(path + '\\' + file, 'r', encoding='utf-8') as file:
                for line in file:
                    self.sentences.append(line.strip())

    
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        transcript = self.sentences[idx]
        return transcript


def load_jfleg_dataset(path='data\\'):
    jfleg_dataset = load_dataset("jfleg", "test", split="test[:1%]", cache_dir=path)
    jfleg_Dataloader = DataLoader(jfleg_dataset, batch_size=1)
    
    print("Number of samples:", len(jfleg_dataset))

    sample_meta0 = jfleg_Dataloader.dataset[0]
    print("""
    Transcript n°0 :

    Path audio: {}
    Sentence: {}
    Corrections: {}
    """.format(path, sample_meta0['sentence'], sample_meta0['corrections']))
    
    return jfleg_Dataloader

def process_jfleg(data, model='T5'):
    sentence = data['sentence']
    corrections = data['corrections']
    
    return sentence[0], 0


def load_ASR_dataset(path='data\\ASR', dataset=ASR_Dataset):
    ASR_Dataset = dataset(path)
    ASR_Dataloader = DataLoader(ASR_Dataset, batch_size=1)
    
    print("Number of samples:", len(ASR_Dataset))
    
    sample_meta0 = ASR_Dataset
    print("""
    Transcript n°0 :
    
    Transcript : {}
    """.format(*sample_meta0))
    
    return ASR_Dataloader

def process_ASR(data, model='T5'):
    return data[0], 0


In [118]:
def GEC(model='T5', output_folder='output', dataset='jfleg'):
    correction_folder = output_folder + "/" + model + '_' + dataset
    os.makedirs(correction_folder, exist_ok=True)
    
    #TQDM loader
    try:
        function = "load_" + dataset + "_dataset"
        dataloader = eval(function)()
        dataloader_tqdm = tqdm(dataloader, total=len(dataloader))
        
        process_function = "process_" + dataset
    except Exception as e:
        raise ValueError(f'Unknown dataset {dataset}{e}')
    
    print(f'Generating corrections for {dataset} with {model} in {correction_folder}')
    if model == 'T5':
        T5 = HappyTextToText("T5", "vennify/t5-base-grammar-correction")
        args = TTSettings(num_beams=5, min_length=1)
    
    with open(correction_folder + '/output.txt', 'w', encoding='utf-8') as file:
        file.write('')
 
    for i, data in enumerate(dataloader_tqdm):
        (sentence, out) = eval(process_function)(data, model=model)
        if out == 1:
            dataloader_tqdm.set_postfix({'status': 'Skipped', 'ID': i})
            continue
            
        try:
            #Passage du transcript dans le modèle
            if model == 'T5':
                result = T5.generate_text("grammar:" + sentence, args=args)
                with open(correction_folder + '/output.txt', 'a', encoding='utf-8') as file:
                    file.write(result.text + '\n')
            elif model == 'BART':
                pass
            else:
                raise ValueError(f'Unknown model {model}')
            
            dataloader_tqdm.set_postfix()

        except Exception as e:
            dataloader_tqdm.set_postfix({'status': 'Error', 'ID': i})
            raise e 
        
        # Validation and test with JFLEG



In [115]:
GEC(model='T5', output_folder='output', dataset='ASR')

Number of samples: 10

    Transcript n°0 :
    
    Transcript : She don't like to eat vegetables.
    


  0%|          | 0/10 [00:00<?, ?it/s]

Generating corrections for ASR with T5 in output/T5_ASR


12/03/2023 16:43:46 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
12/03/2023 16:43:46 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/03/2023 16:43:47 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


In [119]:
GEC(model='T5', output_folder='output', dataset='jfleg')


Number of samples: 7

    Transcript n°0 :

    Path audio: data\
    Sentence: New and new technology has been introduced to the society .
    Corrections: ['New technology has been introduced to society .', 'New technology has been introduced into the society .', 'Newer and newer technology has been introduced into society .', 'Newer and newer technology has been introduced to the society .']
    


  0%|          | 0/7 [00:00<?, ?it/s]

Generating corrections for jfleg with T5 in output/T5_jfleg


12/03/2023 16:44:26 - INFO - happytransformer.happy_transformer -   Using device: cuda:0
12/03/2023 16:44:26 - INFO - happytransformer.happy_transformer -   Moving model to cuda:0
12/03/2023 16:44:27 - INFO - happytransformer.happy_transformer -   Initializing a pipeline


___
## Autres tests

In [16]:
# T5 test:
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config
import torch
#!pip3 install sentencepiece

model_name = "Unbabel/gec-t5_small"
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained(model_name).to("cuda")

text = "My name be Sylvain and I live in Paris, I to 25 year old."
preprocess_text = text.strip().replace("\n","")
print ("original text preprocessed: \n", preprocess_text)

tokenized_text = tokenizer.encode("gec: " + preprocess_text, return_tensors="pt", max_length=128).to("cuda")

# summmarize
summary_ids = model.generate(tokenized_text).to("cuda")

output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print ("\n\nGEC text: \n",output)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


original text preprocessed: 
 My name be Sylvain and I live in Paris, I to 25 year old.


GEC text: 
 My name is Sylvain and I live in Paris. I am 25 years old.


Jfleg training:
https://huggingface.co/vennify/t5-base-grammar-correction


In [13]:
model_b = T5ForConditionalGeneration.from_pretrained('t5-small').to("cuda")
stsb_sentence_1 = preprocess_text
stsb_sentence_2 = output
input_ids = tokenizer.encode("stsb sentence 1: "+stsb_sentence_1+" sentence 2: "+stsb_sentence_2, return_tensors="pt").to("cuda")
stsb_ids = model_b.generate(input_ids)
stsb = tokenizer.decode(stsb_ids[0],skip_special_tokens=True)
print(stsb)


5.0
